In [25]:
spark.sql(
    """
   SHOW tables from PL7G1_GOLD
   """
).toPandas()


namespace                 tableName  isTemporary
0  pl7g1_gold         air_quality_gases        False
1  pl7g1_gold  air_quality_gases_presto        False
2  pl7g1_gold         instalacoescounty        False
3  pl7g1_gold  instalacoescounty_presto        False

In [25]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_GOLD.air_quality_fabrica
    """
)

DataFrame[]

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [27]:
air_quality = spark.table("PL7G1_GOLD.Air_Quality_Gases")
instalacoes = spark.table("PL7G1_GOLD.InstalacoesCounty")

In [28]:
instalacoes_Ar = instalacoes.join(air_quality, (instalacoes.Year == air_quality.Year) & (instalacoes.Borough == air_quality.Borough),"inner").drop(instalacoes.Borough)

In [29]:
instalacoes_Ar =instalacoes_Ar.drop(air_quality.Year)

In [30]:
instalacoes_Ar.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Facility_Name: string (nullable = true)
 |-- SUM_VOC: double (nullable = true)
 |-- AVG_VOC: double (nullable = true)
 |-- MAX_VOC: double (nullable = true)
 |-- MIN_VOC: double (nullable = true)
 |-- SUM_NOx: double (nullable = true)
 |-- AVG_NOx: double (nullable = true)
 |-- MAX_NOx: double (nullable = true)
 |-- MIN_NOx: double (nullable = true)
 |-- SUM_CO: double (nullable = true)
 |-- AVG_CO: double (nullable = true)
 |-- MAX_CO: double (nullable = true)
 |-- MIN_CO: double (nullable = true)
 |-- SUM_CO2: double (nullable = true)
 |-- AVG_CO2: double (nullable = true)
 |-- MAX_CO2: double (nullable = true)
 |-- MIN_CO2: double (nullable = true)
 |-- SUM_Particulates: double (nullable = true)
 |-- AVG_Particulates: double (nullable = true)
 |-- MAX_Particulates: double (nullable = true)
 |-- MIN_Particulates: double (nullable = true)
 |-- SUM_PM10: double (nullable = true)
 |-- AVG_PM10: double (nullable = true)
 |-- MAX_PM10: double

In [31]:
spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.air_quality_fabrica (
        Year INT,
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double,
        Facility_Name VARCHAR(100),
        SUM_VOC Double,
        AVG_VOC Double,
        MAX_VOC Double,
        MIN_VOC Double,
        SUM_NOx Double,
        AVG_NOx Double,
        MAX_NOx Double,
        MIN_NOx Double,
        SUM_CO Double,
        AVG_CO Double,
        MAX_CO Double,
        MIN_CO Double,
        SUM_CO2 Double,
        AVG_CO2 Double,
        MAX_CO2 Double,
        MIN_CO2 Double,
        SUM_Particulates Double,
        AVG_Particulates Double,
        MAX_Particulates Double,
        MIN_Particulates Double,
        SUM_PM10 Double,
        AVG_PM10 Double,
        MAX_PM10 Double,
        MIN_PM10 Double,
        SUM_PM2_5 Double,
        AVG_PM2_5 Double,
        MAX_PM2_5 Double,
        MIN_PM2_5 Double,
        SUM_HAPS Double,
        AVG_HAPS Double,
        MAX_HAPS Double,
        MIN_HAPS Double,
        SUM_SO2 Double,
        AVG_SO2 Double,
        MAX_SO2 Double,
        MIN_SO2 Double,
       longitude VARCHAR(20),
       latitude VARCHAR(20)      
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/air_quality_fabrica'
    """
)

DataFrame[]

In [32]:
instalacoes_Ar = instalacoes_Ar \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true")\
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/air_quality_fabrica")

In [33]:
spark.table("PL7G1_GOLD.air_quality_fabrica").toPandas()

Year        Borough  PM2_5_SUM  PM2_5_AVG  PM2_5_MIN  PM2_5_MAX  NO2_SUM  \
0    2011         Queens     950.19   9.597879       6.72      12.85  1980.45   
1    2011          Bronx     752.10  10.445833       8.81      12.46  1535.94   
2    2011       Brooklyn    1234.44  10.036098       7.86      13.48  2617.66   
3    2010         Queens    1107.63  11.188182       8.44      13.98  2247.61   
4    2011  Staten Island     223.16   9.298333       8.04      11.31   371.35   
..    ...            ...        ...        ...        ...        ...      ...   
950  2019      Manhattan     717.75   8.861111       6.93      12.55  1903.95   
951  2019       Brooklyn     933.53   7.589675       5.98       9.99  2340.29   
952  2019       Brooklyn     933.53   7.589675       5.98       9.99  2340.29   
953  2019         Queens     723.81   7.311212       5.59       9.73  1725.98   
954  2019          Bronx     527.34   7.324167       6.33       9.00  1267.92   

       NO2_AVG  NO2_MIN  NO2_MAX  ...  SUM_HAPS  AVG_HAPS  MAX_HAPS  MIN_HAPS  \
0    20.004545     9.81    26.72  ...       0.0       0.0       0.0       0.0   
1    21.332500    16.19    27.06  ...       0.0       0.0       0.0       0.0   
2    21.281789    12.93    27.75  ...       0.0       0.0       0.0       0.0   
3    22.703131     9.96    33.08  ...       0.0       0.0       0.0       0.0   
4    15.472917     8.32    22.30  ...       0.0       0.0       0.0       0.0   
..         ...      ...      ...  ...       ...       ...       ...       ...   
950  23.505556    13.59    36.18  ...       0.0       0.0       0.0       0.0   
951  19.026748     9.24    29.38  ...       0.0       0.0       0.0       0.0   
952  19.026748     9.24    29.38  ...       0.0       0.0       0.0       0.0   
953  17.434141     7.22    28.09  ...       0.0       0.0       0.0       0.0   
954  17.610000    10.37    26.15  ...       0.0       0.0       0.0       0.0   

     SUM_SO2  AVG_SO2  MAX_SO2  MIN_SO2    longitude    latitude  
0       0.02     0.02     0.02     0.02   -73.817302  40.759776   
1       1.39     1.39     1.39     1.39   -73.920265  40.826153   
2       0.28     0.28     0.28     0.28   -73.990996  40.692529   
3       0.65     0.65     0.65     0.65   -73.817302  40.759776   
4       5.29     5.29     5.29     5.29   -74.076203  40.643498   
..       ...      ...      ...      ...          ...         ...  
950     0.44     0.44     0.44     0.44   -74.007228  40.713054   
951     0.00     0.00     0.00     0.00   -73.990996  40.692529   
952     0.00     0.00     0.00     0.00   -73.990996  40.692529   
953     0.13     0.13     0.13     0.13   -73.817302  40.759776   
954    11.62    11.62    11.62    11.62   -73.920265  40.826153   

[955 rows x 57 columns]

In [34]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/air_quality_fabrica/`
""").show()

++
||
++
++



In [35]:
spark.sql("""
    CREATE EXTERNAL TABLE IF NOT EXISTS PL7G1_GOLD.air_quality_fabrica_presto (
        Year INT,
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double,
        Facility_Name VARCHAR(100),
        SUM_VOC Double,
        AVG_VOC Double,
        MAX_VOC Double,
        MIN_VOC Double,
        SUM_NOx Double,
        AVG_NOx Double,
        MAX_NOx Double,
        MIN_NOx Double,
        SUM_CO Double,
        AVG_CO Double,
        MAX_CO Double,
        MIN_CO Double,
        SUM_CO2 Double,
        AVG_CO2 Double,
        MAX_CO2 Double,
        MIN_CO2 Double,
        SUM_Particulates Double,
        AVG_Particulates Double,
        MAX_Particulates Double,
        MIN_Particulates Double,
        SUM_PM10 Double,
        AVG_PM10 Double,
        MAX_PM10 Double,
        MIN_PM10 Double,
        SUM_PM2_5 Double,
        AVG_PM2_5 Double,
        MAX_PM2_5 Double,
        MIN_PM2_5 Double,
        SUM_HAPS Double,
        AVG_HAPS Double,
        MAX_HAPS Double,
        MIN_HAPS Double,
        SUM_SO2 Double,
        AVG_SO2 Double,
        MAX_SO2 Double,
        MIN_SO2 Double,
       longitude VARCHAR(20),
       latitude VARCHAR(20)  
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
    STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/air_quality_fabrica/_symlink_format_manifest/'
    """).show()

++
||
++
++



In [ ]:

#.select("Year", "Borough", "PM2_5_SUM", "PM2_5_AVG", "PM2_5_MIN", "PM2_5_MAX", "SUM_PM2_5", "AVG_PM2_5", "MAX_PM2_5", "MIN_PM2_5", "SUM_SO2", "AVG_SO2", "MAX_SO2", "MIN_SO2", "SO2_SUM", "SO2_AVG", "SO2_MIN", "SO2_MAX")